Notebook for phrase score interpolation with DP and phrase expansion scoring

In [ ]:
%matplotlib inline

import sys 
import os 

nb_dir = os.getcwd()
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

from plotlib.loaders import *
from plotlib.plotters import *

from phdconf import config 
from phdconf.config import *

In [ ]:
queries = load_queries(config.AUS_TOPIC_PATH)
broad, specific = load_query_types(queries)

In [ ]:
index_name = 'filtered-phrasestop'#, config.SIGIR_INDEX_NAME]
qrel_paths =  [config.AUS_QREL_PATH]#, config.SIGIR_QREL_PATH]
rel_levels = [config.AUS_REL_LEVEL]#, config.SIGIR_REL_LEVEL]
display_names = ['AUS']#, 'SIGIR']

In [ ]:
base_df = load_1d_dfs([index_name], qrel_paths, os.path.join('/home/danlocke/phd-generated', 'preprocessing', 'dirichlet_prior'),'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, 1050, 1050, 1)[0][0]
base_qry = load_1d_dfs([index_name], qrel_paths, os.path.join('/home/danlocke/phd-generated', 'preprocessing', 'dirichlet_prior'),'case-topics-{0}-unigram_dir_mu_{1:.2f}.run', rel_levels, 1050, 1050, 1, per_query=True)[0][0]

## Phrase scoring

In [ ]:
names = ['nnphrase', 'nnphrase-exp', 'phrase-exp-qry', 'nnphrase-exp-qry']
inter = Interpolater(os.path.join(BASE_DIR, 'preprocessing', 'dirichlet_prior', 'case-topics-filtered-phrasestop-unigram_dir_mu_1050.00.run'), normalize=True)

_dir = '/home/danlocke/go/src/github.com/dan-locke/phd/phrase-expansion/'

In [ ]:
dfs = []
for n in names: 
    interped_dfs = []
    for _lambda in np.arange(0, 1.0, 0.01):
        inter.interpolate(os.path.join(_dir, 'case-topics-{0}-{1}.run'.format(index_name, n)), _lambda, 'tmp.run')
        interped_dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'])[0])
    dfs.append(interped_dfs)

In [ ]:
qry_dfs = []
for n in names: 
    interped_dfs = []
    for _lambda in np.arange(0, 1.0, 0.01):
        inter.interpolate(os.path.join(_dir, 'case-topics-{0}-{1}.run'.format(index_name, n)), _lambda, 'tmp.run')
        interped_dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'], per_query=True)[0])
    qry_dfs.append(interped_dfs)

In [ ]:
metrics = copy.copy(config.METRIC_NAMES)
del metrics['recall_100']

In [ ]:


to = 50

plot = plot_tune_1d_comp(['base', 'phrase', 'phrase-exp', 'phrase-qry', 'phrase-exp-qry'], metrics, [[base_df for x in range(to+1)]] + [x[:to+1] for x in dfs], 0.00, (to)/100, 0.01, legend_x=0.991, styles=['--'], ylims=ylims)
# plot.savefig('figures/ausnl-phrase-expansion-interp.pdf')

In [ ]:
# base_sdm = load_1d_dfs([index_name], qrel_paths, os.path.join('/home/danlocke/phd-generated', 'sdm_rerank'), 'case-topics-filtered-phrasestop-sdm_rerank-smooth-dir-mu-{1:.2f}-weights-0.80-0.05-0.15-window-6.run', rel_levels, 1050, 1050, 1)[0][0]

plot = plot_tune_1d_comp(['base', 'phrase'], metrics, [[base_df for x in range(to+1)]] + [dfs[0][:to+1]], 0.00, (to)/100, 0.01, legend_x=0.965, styles=['--'])
# plot.savefig('figures/ausnl-phrase-interp.pdf')

In [ ]:
tt_folds = read_folds('ausnl-folds.txt')

In [ ]:
stop_df = pd.DataFrame(columns=metrics)

for ab, runs in zip(['phrase', 'phrase-exp', 'phrase-qry', 'phrase-exp-qry'], qry_dfs):
    cross = cross_validation(runs, tt_folds, metrics, base_qry)
    stop_df.loc[ab] = cross[0]

In [ ]:
stop_df.loc['$R$'] = base_df.round(4)

In [ ]:
# stop_df = stop_df.rename(index={'nnphrase': 'phrase', 'nnphrase-exp': 'phrase-exp', 'phrase-exp-qry': 'phrase-qry', 'nnphrase-exp-qry': 'phrase-exp-qry'})
stop_df = stop_df.reindex(['$R$', 'phrase', 'phrase-exp', 'phrase-qry', 'phrase-exp-qry'])
stop_df

In [ ]:
# write_table('tables/ausnl-phrase', bold_max(stop_df.reindex(['$R$', 'phrase'])).rename(columns=metrics).drop('Unjudged@20',axis='columns').to_latex(escape=False))

In [ ]:
# write_table('tables/ausnl-phrase-exp', bold_max(stop_df).rename(columns=metrics).drop('Unjudged@20',axis='columns').to_latex(escape=False))